In [4]:
from tensorflow.keras.applications import InceptionV3, VGG16, ResNet50, MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
import keras
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import cv2
import os
from tensorflow.keras.preprocessing import image
import random

In [5]:
root_path="data/"
ngantuk_path=root_path+"/ngantuk/wajah"
sadar_path=root_path+"/sadar/wajah"
chekpoints_path = "Model CNN/Checkpoint"
model_path="Model CNN/"

In [6]:
# insert all image in dataset folder to list
ngantuks = []
sadars = []
for filename in os.listdir(ngantuk_path):
    if filename.endswith(".jpg") or filename.endswith(".png"):
      image = cv2.imread(os.path.join(ngantuk_path, filename))
      ngantuks.append(image)
      print(filename)

for filename in os.listdir(sadar_path):
    if filename.endswith(".jpg") or filename.endswith(".png"):
      image = cv2.imread(os.path.join(sadar_path, filename))
      sadars.append(image)
      print(filename)

ngantuk_0.jpg
ngantuk_1.jpg
ngantuk_10.jpg
ngantuk_100.jpg
ngantuk_1000.jpg
ngantuk_1001.jpg
ngantuk_1002.jpg
ngantuk_1003.jpg
ngantuk_1004.jpg
ngantuk_1005.jpg
ngantuk_1006.jpg
ngantuk_1007.jpg
ngantuk_1008.jpg
ngantuk_1009.jpg
ngantuk_101.jpg
ngantuk_1010.jpg
ngantuk_1011.jpg
ngantuk_1012.jpg
ngantuk_1013.jpg
ngantuk_1014.jpg
ngantuk_1015.jpg
ngantuk_1016.jpg
ngantuk_1017.jpg
ngantuk_1018.jpg
ngantuk_1019.jpg
ngantuk_102.jpg
ngantuk_1020.jpg
ngantuk_1021.jpg
ngantuk_1022.jpg
ngantuk_1023.jpg
ngantuk_1024.jpg
ngantuk_1025.jpg
ngantuk_1026.jpg
ngantuk_1027.jpg
ngantuk_1028.jpg
ngantuk_1029.jpg
ngantuk_103.jpg
ngantuk_1030.jpg
ngantuk_1031.jpg
ngantuk_1032.jpg
ngantuk_1033.jpg
ngantuk_1034.jpg
ngantuk_1035.jpg
ngantuk_1036.jpg
ngantuk_1037.jpg
ngantuk_1038.jpg
ngantuk_1039.jpg
ngantuk_104.jpg
ngantuk_1040.jpg
ngantuk_1041.jpg
ngantuk_1042.jpg
ngantuk_1043.jpg
ngantuk_1044.jpg
ngantuk_1045.jpg
ngantuk_1046.jpg
ngantuk_1047.jpg
ngantuk_1048.jpg
ngantuk_1049.jpg
ngantuk_105.jpg
ngantuk_105

In [7]:
rotates=[cv2.ROTATE_90_CLOCKWISE,cv2.ROTATE_90_COUNTERCLOCKWISE]
# resize and input data, and target
img_size=200
data=[]
hitung=0
target=[]
for img1 in ngantuks:
  img=img1.copy()
  try:
    resized=cv2.resize(img,(img_size,img_size))
    rotated=cv2.rotate(resized,rotates[random.randint(0,1)])
    fliped=cv2.flip(resized,random.randint(0,1))
    data.append(resized)
    target.append("ngantuk")
    hitung+=1
    data.append(rotated)
    target.append("ngantuk")
    hitung+=1
    data.append(fliped)
    target.append("ngantuk")
    hitung+=1
  except Exception as e:
    print('Exception:',e)

for img1 in sadars:
  try:
    img=img1.copy()
    resized=cv2.resize(img,(img_size,img_size))
    rotated=cv2.rotate(resized,rotates[random.randint(0,1)])
    fliped=cv2.flip(resized,random.randint(0,1))
    data.append(resized)
    target.append("sadar")
    hitung+=1
    data.append(rotated)
    target.append("sadar")
    hitung+=1
    data.append(fliped)
    target.append("sadar")
    hitung+=1
  except Exception as e:
    print('Exception:',e)

print(str(len(ngantuks)+len(sadars)))
print(str(hitung))

14202
42606


In [8]:
# encode kategori (target) jadi angka
target2=target.copy()
data2=data.copy()
encoder = LabelEncoder()
target=encoder.fit_transform(target)
target = keras.utils.to_categorical(target, 2)
# split data
xtrain,xtest,ytrain,ytest=train_test_split(data,target,test_size=0.2,random_state=1)

In [9]:
# MobileNetV2
base_model_mobilenet = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
# InceptionV3
base_model_inception = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
# VGG16
base_model_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
# ResNet50
base_model_resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

C:\Users\nicky\AppData\Local\Temp\ipykernel_1552\771792798.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model_mobilenet = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))


In [10]:
model_mobilenet=keras.models.load_model(model_path+"model_mobilenetv2.keras")
model_inception=keras.models.load_model(model_path+"model_inception_transfer.keras")
model_resnet=keras.models.load_model(model_path+"model_resnet50_tf.keras")
model_vgg16=keras.models.load_model(model_path+"model_vgg16_tf.keras")

In [11]:
container=0
i=0
for img_check in data2:
  print("data ke : "+str(i))
  x = image.img_to_array(img_check)
  x = np.expand_dims(x, axis=0)

  # Make a prediction
  preds = model_mobilenet.predict(x)
  hasil=""
  
  # Get the predicted class
  predicted_class = np.argmax(preds)
  if predicted_class == 0:
    hasil="ngantuk"
  else:
    hasil="sadar"
  if hasil != target2[i]:
    container+=1
  i+=1

data ke : 0


AttributeError: 'numpy.ndarray' object has no attribute 'img_to_array'

In [ ]:
print(str(container))

In [ ]:
model_mobilenet_notop = tf.keras.models.Model(inputs=model_mobilenet.input, outputs=model_mobilenet.layers[-2].output)
features_train = model_mobilenet_notop.predict(xtrain2)
features_test = model_mobilenet_notop.predict(xtest2)
if features_train.ndim == 2:
    features_train = features_train.reshape(features_train.shape[0], 1, 1, features_train.shape[1])
    features_test = features_test.reshape(features_test.shape[0], 1, 1, features_test.shape[1])

In [12]:
xtrain2 = np.array(xtrain.copy())
xtest2 = np.array(xtest.copy())
ytrain2 = np.array(ytrain.copy())
ytest2 = np.array(ytest.copy())
loss,acc=model_mobilenet.evaluate(xtest2, ytest2)
print("mobilenet")
print("loss: ",loss)
print("acc: ",acc)
print("")

print("vgg16")
loss,acc=model_vgg16.evaluate(xtest2, ytest2)
print("loss: ",loss)
print("acc: ",acc)
print("")

print("inception")
loss,acc=model_inception.evaluate(xtest2, ytest2)
print("loss: ",loss)
print("acc: ",acc)
print("")

print("resnet")
loss,acc=model_resnet.evaluate(xtest2, ytest2)
print("loss: ",loss)
print("acc: ",acc)

 44/267 ━━━━━━━━━━━━━━━━━━━━ 15:55 4s/step - accuracy: 0.9910 - loss: 0.0211

KeyboardInterrupt: 